# Test API inference service

In [1]:
import requests

In [2]:
model_name='income-model'
namespace='explainable-ml-engine-demo'
host=model=model_name+'.'+namespace+'.example.com'
ingress_host='localhost'
port=8080

In terminal:
```bash
INGRESS_GATEWAY_SERVICE=$(kubectl get svc --namespace istio-system --selector="app=istio-ingressgateway" --output jsonpath='{.items[0].metadata.name}')
kubectl port-forward --namespace istio-system svc/${INGRESS_GATEWAY_SERVICE} 8080:80
```

In [3]:
inference_service_url='http://'+ingress_host+':'+str(port)+'/v1/models/'+model_name
inference_service_url

'http://localhost:8080/v1/models/income-model'

In [4]:
inference_service_url_predict=inference_service_url+':predict'
inference_service_url_predict

'http://localhost:8080/v1/models/income-model:predict'

In [5]:
inference_service_url_explain=inference_service_url+':explain'
inference_service_url_explain

'http://localhost:8080/v1/models/income-model:explain'

In [6]:
model_input = {
  "instances":[
    [39, 7, 1, 1, 1, 1, 4, 1, 2174, 0, 40, 9]
    ]
  }

In [7]:
respons=requests.post(inference_service_url_explain, headers={'Host': host}, json=model_input)

In [90]:
respons.json()

{'meta': {'name': 'AnchorTabular',
  'type': ['blackbox'],
  'explanations': ['local'],
  'params': {'seed': None,
   'disc_perc': [25, 50, 75],
   'threshold': 0.95,
   'delta': 0.1,
   'tau': 0.15,
   'batch_size': 100,
   'coverage_samples': 10000,
   'beam_size': 1,
   'stop_on_first': False,
   'max_anchor_size': None,
   'min_samples_start': 100,
   'n_covered_ex': 10,
   'binary_cache_size': 10000,
   'cache_margin': 1000,
   'verbose': False,
   'verbose_every': 1,
   'kwargs': {}}},
 'data': {'anchor': ['Hours per week <= 40.00',
   'Capital Gain <= 0.00',
   'Capital Loss <= 0.00',
   'Workclass = Self-emp-not-inc'],
  'precision': 0.9113082039911308,
  'coverage': 0.7055333333333333,
  'raw': {'feature': [10, 8, 9, 1, 10],
   'mean': [0.8201595358955766,
    0.8565768621236133,
    0.8736401673640167,
    0.8872180451127819,
    0.9113082039911308],
   'precision': [0.8201595358955766,
    0.8565768621236133,
    0.8736401673640167,
    0.8872180451127819,
    0.911308203991

In [9]:
respons.json()['data']['precision']

0.9623824451410659

In [10]:
respons.json()['data']['coverage']

0.3286

In [39]:
from alibi.datasets import fetch_adult
import json

In [12]:
adult=fetch_adult()

In [102]:
person = adult.data[239]

In [103]:
def jsonfy_peron(features, data, category_map):
    json_data={}
    j=0
    for i in range(0,len(data)-1):
        if i in category_map.keys():
            json_data[features[i]]=category_map[i][data[list(category_map.keys())[j]]]
            j=j+1
        else:
            json_data[features[i]]=data[i]
    return json_data

In [104]:
jsonfy_peron(adult.feature_names, person, adult.category_map)

{'Age': 23,
 'Workclass': 'Private',
 'Education': 'High School grad',
 'Marital Status': 'Never-Married',
 'Occupation': 'Blue-Collar',
 'Relationship': 'Own-child',
 'Race': 'White',
 'Sex': 'Male',
 'Capital Gain': 0,
 'Capital Loss': 0,
 'Hours per week': 40}

In [105]:
model_input = {
  "instances":[
    person.tolist()
    ]
  }

In [106]:
respons=requests.post(inference_service_url_explain, headers={'Host': host}, json=model_input)

In [107]:
print(respons.json()['data']['raw']['prediction'])
print(respons.json()['data']['anchor'])
print(respons.json()['data']['precision'])
print(respons.json()['data']['coverage'])


0
['Marital Status = Never-Married', 'Relationship = Own-child']
1.0
0.3286
